In [1]:
import pandas as pd
import joblib

from collections import defaultdict
from Levenshtein import distance
from sklearn.feature_extraction.text import CountVectorizer
from input_parsing.parsing import BBCRecipesParses
from data_preprocessing.preprocessing import DataPreprocessing
from recipe_model.tf_idf import TF_IDF_RecipeRecommendation
from recipe_model.word2vec import MeanWord2Vec, TfIdfWord2Vec
from recipe_model.doc2vec import CustomDoc2Vec
from constants import DATA_PATH_FULL_CSV, DATA_PARSED_PATH_CSV, DOC2VEC_MODEL, DATA_PARSED_PATH_PICKLE, DATA_PATH_FULL_PICKLE, DROP_DUPLICATES_BY_COLUMN, INGREDIENTS_PARSED_COLUMN

### parse data from https://www.bbcgoodfood.com


In [2]:
# parser = BBCRecipesParses()


save List[RecipeModel] in pickle

In [3]:
# data = parser.parse_recipes(return_dataframe=False)
# joblib.dump(data, DATA_PATH_FULL_PICKLE)
# data = joblib.load(DATA_PATH_FULL_PICKLE)
# data

save data in scv


### Везде работать с List[RecipeModel] ?

In [4]:
# data = parser.parse_recipes(return_dataframe=True)
# df = pd.DataFrame(data)
# df.to_csv(
#     DATA_PATH_FULL_CSV,
#     sep="\t", index=False)

### preprocess and save dataframe (CSV) 


read data, drop duplicates

In [5]:
df = pd.read_csv(DATA_PATH_SHORT_CSV, sep='\t')
recipe_df = df.copy()
print(recipe_df.shape)
recipe_df.sort_values(DROP_DUPLICATES_BY_COLUMN, inplace=True)
recipe_df.drop_duplicates(subset=DROP_DUPLICATES_BY_COLUMN, keep='first', inplace=True)
recipe_df.reset_index(drop=True, inplace=True)
# print(recipe_df)


(1520, 8)


preprocess to df

In [6]:
dt_preprocess = DataPreprocessing()
dt_preprocessed = dt_preprocess.preprocess_df(recipe_df)
# dt_preprocessed

save df

In [7]:
dt_preprocessed.to_csv(DATA_PARSED_PATH_CSV, sep="\t", index=False)

### preprocess structured data (PICKLE)


read (from csv, more data was loaded to csv) data, drop duplicates 

In [8]:
df = pd.read_csv(DATA_PATH_FULL_CSV, sep='\t')
recipe_df = df.copy()
recipes_list = list(recipe_df.itertuples(name='ExtendedRecipeModel', index=False))
unique_recipes_list = list(set(recipes_list))

preprocess to list of objects

In [9]:
preprocessed_list = dt_preprocess.preprocess_list(unique_recipes_list)

save list

In [10]:
joblib.dump(preprocessed_list, DATA_PARSED_PATH_PICKLE)
# data = joblib.load(DATA_PARSED_PATH_PICKLE)
# data

['/Users/arina/study/ds/project/food_recommendation/bbc_goodfood/project_food/data/structured_data/data_goodfood_bbc_parsed.pickle']

test user input

In [11]:
test_string = "[' 22(del)5g unsalted          butter,  (delete) softened', '225g caster sugar', '4 eggs', " \
                "'225g self,-raising flour', '1        lemon, zested', '1½ lemons, juiced', '85g caster sugar',' salt']"
list_test_ingredients = dt_preprocess.preprocess_request(test_string)
# list_test_ingredients

### Check if there are similar words
#### Using csv data

In [12]:
def get_cv_words_similarity(cv_table) -> dict:
    similarity_scores = defaultdict(dict)
    for i in range(len(cv_table)):
        similarity_scores[cv_table[i]] = {}
        for j in range(i + 1, len(cv_table)):
            d = distance(cv_table[i], cv_table[j])
            similarity_scores[cv_table[i]][cv_table[j]] = d
            similarity_scores[cv_table[j]][cv_table[i]] = d
    return similarity_scores

In [13]:
parsed_ingredients_column = dt_preprocessed[INGREDIENTS_PARSED_COLUMN].apply(lambda x: [str(sentence) for sentence in x])
parsed_ingredients_column = parsed_ingredients_column.apply(str)

In [14]:
cv = CountVectorizer(ngram_range=(1, 1), analyzer='word', lowercase=True)
cv_model = cv.fit(parsed_ingredients_column)
scores = get_cv_words_similarity(cv_model.get_feature_names_out())

In [15]:
sorted_scores = {k: sorted(v.items(), key=lambda x: x[1]) for k, v in scores.items()}

for word, scores in sorted_scores.items():
    print(f"current '{word}':")
    for similar_word, score in scores[:3]:
        print(f" - {similar_word} (levenshtein: {score})")
    print()

current 'accord':
 - afford (levenshtein: 2)
 - acid (levenshtein: 3)
 - across (levenshtein: 3)

current 'acid':
 - activ (levenshtein: 2)
 - ad (levenshtein: 2)
 - anis (levenshtein: 2)

current 'across':
 - choos (levenshtein: 3)
 - close (levenshtein: 3)
 - crown (levenshtein: 3)

current 'action':
 - activ (levenshtein: 2)
 - option (levenshtein: 2)
 - altern (levenshtein: 3)

current 'activ':
 - addit (levenshtein: 3)
 - agav (levenshtein: 3)
 - anis (levenshtein: 3)

current 'ad':
 - adam (levenshtein: 2)
 - ale (levenshtein: 2)
 - arg (levenshtein: 2)

current 'adam':
 - agav (levenshtein: 2)
 - atar (levenshtein: 2)
 - away (levenshtein: 2)

current 'addit':
 - adobo (levenshtein: 3)
 - aduki (levenshtein: 3)
 - andor (levenshtein: 3)

current 'adobo':
 - aduki (levenshtein: 3)
 - ancho (levenshtein: 3)
 - andor (levenshtein: 3)

current 'aduki':
 - aïoli (levenshtein: 3)
 - chunki (levenshtein: 3)
 - cooki (levenshtein: 3)

current 'afford':
 - almond (levenshtein: 3)
 - ando

stop words: 

grey, white, five, one, two, three, 


middl, around, hot, high, 


big, irregular, regular,


kept, etc, king, light, luxuri,


made, make, day, metal,


room, nice, mix, flat, possibl, 


visibl, prefer, buy, rich, save, serv, 


want, wash, whatev,white, winter, work, write, young

similar words:


crème vs creme, chilli vs chili, fraich vs fraîch,


halv vs half, hot, harissa vs harrisa,  jalapen vs jalapeño


lengthway vs lenthway


muscovado vs muscavdo, oilv vs oliv


puré vs pure,


smoki vs smoke, soft vs soften vs softenend, start vs starter, tast vs tasti, turkish vs turkey,


yoghurt vs yogurt

(all)
crème vs creme, chilli vs chili, flavourless vs flavour, frai vs fri, fraich vs fraîch, grey, five


middl, around, halv vs half, hot, harissa vs harrisa, high, big, irregular, regular, jalapen vs jalapeño


kept, etc, king, lard vs lardon, lengthway vs lenthway, light, luxuri, made, make, maria, mari


day, metal, muscovado, muscavdo, room, nice, oat vs oatmeal, oilv vs oliv


mix, min, flat, possibl, visibl, hot, prefer, puré vs pure, buy, rich, save, serv, small vs smaller


smoki vs smoke, soft vs soften vs softenend, start vs starter, tast vs tasti, turkish vs turkey, want


wash, whatev, white, winter, work, write, yoghurt vs yogurt, young

### TF-IDF

In [16]:
user_inp = "['cinnamon', 'sugar', 'apple', 'flour', 'butter']"

tf_idf = TF_IDF_RecipeRecommendation.create_instance(from_csv=True, path=DATA_PARSED_PATH_CSV).prepare_model()
# tf_idf = TF_IDF_RecipeRecommendation.create_instance(from_csv=False, path=DATA_PARSED_PATH_PICKLE).prepare_model()
rec = tf_idf.get_recommendations(user_inp)
rec


ingredients
['1kg Bramley apples, peeled, cored and chopped  ', '250g  caster sugar', '½ lemon, juiced ', '1 cinnamon stick']                                                                                                                                                                                                                                                                                                       0.484240
['1kg Bramley apples', '140g golden caster sugar', '½ tsp cinnamon', '3 tbsp flour', '225g butter, room temperature', '50g golden caster sugar, plus extra', '2 eggs', '350g plain flour, preferably organic', 'softly whipped cream, to serve']                                                                                                                                                                        0.477769
['85g butter, melted ', '140g self-raising flour', '100g golden caster sugar', '1 tbsp baking powder', '200ml milk', '1 egg, beaten', '1 tsp vanilla extra

In [17]:
tf_idf.to_pickle()
tf_idf.from_pickle()

### word2vec

##### mean


In [18]:
mean_word_to_vec = MeanWord2Vec.create_instance().train_model()
rec = mean_word_to_vec.get_recommendations(user_inp)
print(rec)

ingredients
['575g Bramley apple (3 medium apples), peeled, cored and sliced to 1cm thick', '2 tbsp golden caster sugar', '175g plain flour', '110g golden caster sugar', '110g cold butter', '1 tbsp rolled oats', '1 tbsp demerara sugar', 'double cream, clotted cream or custard, to serve']                                0.942036
['200g desiccated coconut', '85g caster sugar', '2 eggs, beaten', '200g bar white chocolate, roughly chopped', '85g glacé cherries, halved']                                                                                                                                                                        0.935849
['1kg Bramley apples', '140g golden caster sugar', '½ tsp cinnamon', '3 tbsp flour', '225g butter, room temperature', '50g golden caster sugar, plus extra', '2 eggs', '350g plain flour, preferably organic', 'softly whipped cream, to serve']                                                                    0.931741
['120g plain flour', '60g caster suga

In [19]:
mean_word_to_vec.to_pickle()
mean_word_to_vec.from_pickle()

##### tfidf

In [20]:
tfidf_word_to_vec = TfIdfWord2Vec.create_instance().train_model()
rec = tfidf_word_to_vec.get_recommendations(user_inp)
print(rec)

ingredients
['1kg Bramley apples, peeled, cored and chopped  ', '250g  caster sugar', '½ lemon, juiced ', '1 cinnamon stick']                                                                                                                                                                                                   0.945042
['200g desiccated coconut', '85g caster sugar', '2 eggs, beaten', '200g bar white chocolate, roughly chopped', '85g glacé cherries, halved']                                                                                                                                                                        0.924497
['575g Bramley apple (3 medium apples), peeled, cored and sliced to 1cm thick', '2 tbsp golden caster sugar', '175g plain flour', '110g golden caster sugar', '110g cold butter', '1 tbsp rolled oats', '1 tbsp demerara sugar', 'double cream, clotted cream or custard, to serve']                                0.912621
['115g unsalted butter, diced and chi

In [21]:
tfidf_word_to_vec.to_pickle()
tfidf_word_to_vec.from_pickle()

### doc2vec

In [ ]:
model = CustomDoc2Vec.create_instance().train_model()
rec = model.get_recommendations(user_inp)
print(rec)


#### save model

In [ ]:
model.to_pickle(DOC2VEC_MODEL)